## Getting the data 

In [18]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Indexing the data

In [20]:
from elasticsearch import Elasticsearch

In [21]:
es_client = Elasticsearch('http://localhost:9200') 

In [22]:
print(es_client.info())

{'name': 'd41cee32fff1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'UYNFOCoiRIWRTCLsXCNcXA', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

In [24]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [25]:
from tqdm.auto import tqdm

In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Searching the data

In [27]:
def elastic_search(query: str, n_response:int = 5, course: str | None = None):
    search_query = {
        "size": n_response,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    if course:
        search_query['query']['bool']['filter'] = {'term' : {'course' : course}}

    response = es_client.search(index=index_name, body=search_query)

    return response

In [28]:
query = "How do execute a command on a Kubernetes pod?"

response = elastic_search(query)

print(response)

{'took': 188, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 701, 'relation': 'eq'}, 'max_score': 43.83977, 'hits': [{'_index': 'course-questions', '_id': 'mo52bZcBpCFHczpQBZUt', '_score': 43.83977, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'KY52bZcBpCFHczpQCJYH', '_score': 35.12413, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', '

In [29]:
print("Max score:", response['hits']['max_score'])

Max score: 43.83977


In [30]:
query = "How do copy a file to a Docker container?"

course = "machine-learning-zoomcamp"

n_response = 3

response = elastic_search(query, n_response=n_response, course=course)

import json

print(json.dumps(response.body, indent=2))


{
  "took": 44,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 340,
      "relation": "eq"
    },
    "max_score": 73.101364,
    "hits": [
      {
        "_index": "course-questions",
        "_id": "mo52bZcBpCFHczpQBZUt",
        "_score": 73.101364,
        "_source": {
          "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
          "section": "5. Deploying Machine Learning Models",
          "question": "How do I debug a docker container?",
          "course": "machine-learning-zoomcamp"
        }
      },
      {
        "_index": "course-questions",
        "_id": "uY52bZcBpCFHczpQBZXV",


In [31]:
print(response['hits']['hits'][-1]['_source']['question'])

How do I copy files from a different folder into docker container’s working directory?


## Building a prompt

In [32]:
q4_responses = response['hits']['hits']

print(json.dumps(q4_responses, indent=2))

[
  {
    "_index": "course-questions",
    "_id": "mo52bZcBpCFHczpQBZUt",
    "_score": 73.101364,
    "_source": {
      "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
      "section": "5. Deploying Machine Learning Models",
      "question": "How do I debug a docker container?",
      "course": "machine-learning-zoomcamp"
    }
  },
  {
    "_index": "course-questions",
    "_id": "uY52bZcBpCFHczpQBZXV",
    "_score": 66.51779,
    "_source": {
      "text": "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The ba

In [33]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [34]:
context = "\n\n".join([
    context_template.format(
        question=hit['_source']['question'],
        text=hit['_source']['text']
    )
    for hit in q4_responses
])

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [35]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [36]:
formatted_prompt = prompt_template.format(
    question=query,
    context=context
)

print(formatted_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [37]:
print("Length of formatted prompt:", len(formatted_prompt))

Length of formatted prompt: 1446


## Tokens

In [38]:
import tiktoken

In [39]:
encoding = tiktoken.encoding_for_model("gpt-4o")


In [53]:
prompt_tokens = encoding.encode(formatted_prompt)
print("Number of tokens in the prompt:", len(prompt_tokens))

Number of tokens in the prompt: 320


## Sending requests to LLM

In [41]:
from openai import OpenAI

In [42]:
client = OpenAI()

In [43]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [45]:
print(formatted_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [ ]:
answer = llm(formatted_prompt)
print("Answer from LLM:")
print(answer)

Answer from LLM:
To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


## Price calculation

In [49]:
input_per_1000_tokens_price = 0.005
output_per_1000_tokens_price = 0.015

In [50]:
input_tokens = 150
output_tokens = 250

In [51]:
final_price = 1000 * (input_per_1000_tokens_price / 1000 * input_tokens + output_per_1000_tokens_price / 1000 * output_tokens)
print(f"Final price for the request: ${final_price:.6f}")

Final price for the request: $4.500000


In [52]:
answer_tokens = encoding.encode(answer)
print("Number of tokens in the answer:", len(answer_tokens))

Number of tokens in the answer: 46


In [58]:
final_price = input_per_1000_tokens_price / 1000 * len(prompt_tokens) + output_per_1000_tokens_price / 1000 * len(answer_tokens)
print(f"Final price for our LLM request: ${final_price}")

Final price for our LLM request: $0.00229
